# Uncertainty Analysis with NZUpy's Range Runs

## Overview of Range Runs

NZUpy allows for uncertainty analysis through "range runs," which highlight how uncertainties in the response of gross emissions demand to carbon pricing can affect market outcomes. This notebook demonstrates:

1. How to set up and run 'range runs'
2. Use a custom function to manually edit input data
3. Generate range charts with uncertainty bands
4. Present results using inbuilt NZUpy features

Let's begin by importing the necessary libraries and setting up our model.

## 1. Set Up the NZUpy Model

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from pathlib import Path
import sys
import os

#Set up NZUpy and chart generation
project_root = Path().absolute().parent
sys.path.insert(0, str(project_root))
from model.core.base_model import NZUpy
from model.utils.chart_generator import ChartGenerator
from model.interface.chart_display import create_chart_page

# Set our input and output directories
data_dir = project_root / "data"
output_dir = project_root / "examples" / "outputs" / "03_range_runs"
os.makedirs(output_dir, exist_ok=True)

In [2]:
# Initialise NZUpy
NZU = NZUpy(data_dir=data_dir)

## 2. Set Up Range Scenario Type

For uncertainty analysis, we need to set the model to use "Range" scenario type, which automatically configures the model to run with multiple demand sensitivities.


In [3]:
# Define time periods: start year, end year
NZU.define_time(2024, 2050)

# Set the scenario type to 'Range' for uncertainty analysis
NZU.define_scenario_type('Range')

# Prime the model - this will automatically set up the 5 sensitivity levels
NZU.prime()

Time periods defined:
  Optimisation: 2024-2050
Scenario type set to 'Range'
Model primed with 5 scenarios:
  [0] 95% Lower
  [1] 1 s.e lower
  [2] central
  [3] 1 s.e upper
  [4] 95% Upper


## 3. Configure Policy Scenario

We'll set up our Range run to examine a topical policy scenario in which the Government actively pursues reform to supply for industrial allocation and forestry removals. We'll do this by:

1. Loading the Goverment's 'low' projection for forestry removals.
2. Using a custom function to adjust default industrial allocation projections.

In [4]:
# Our approach to configuring the range run is similar to that for single scenario runs.
NZU.configure_range_scenarios()

# Apply the low forestry config to all sensitivity level scenarios
for i in range(len(NZU.scenarios)):
    NZU.use_config(i, 'forestry', 'low')
    print(f"Applied 'low' forestry configuration to scenario {i} ({NZU.scenarios[i]})")

# Verify forestry configuration for each sensitivity level
print("\nVerifying forestry configuration for each sensitivity level:")
for i, scenario in enumerate(NZU.scenarios):
    config = NZU.component_configs[i]
    print(f"Scenario {i} ({scenario}) forestry config: {config.forestry}")


[DEBUG] Starting range scenario configuration
[DEBUG] Storing existing component configurations
[DEBUG] Stored config for scenario 0: {'forestry': 'central', 'auctions': 'central', 'industrial_allocation': 'central', 'emissions': 'central', 'stockpile': 'central', 'demand_sensitivity': 'central', 'demand_model_number': 2}
[DEBUG] Stored config for scenario 1: {'forestry': 'central', 'auctions': 'central', 'industrial_allocation': 'central', 'emissions': 'central', 'stockpile': 'central', 'demand_sensitivity': 'central', 'demand_model_number': 2}
[DEBUG] Stored config for scenario 2: {'forestry': 'central', 'auctions': 'central', 'industrial_allocation': 'central', 'emissions': 'central', 'stockpile': 'central', 'demand_sensitivity': 'central', 'demand_model_number': 2}
[DEBUG] Stored config for scenario 3: {'forestry': 'central', 'auctions': 'central', 'industrial_allocation': 'central', 'emissions': 'central', 'stockpile': 'central', 'demand_sensitivity': 'central', 'demand_model_num

In [5]:
# Get the current industrial allocation data
industrial_data = NZU.data_handler.get_industrial_allocation_data(config='central')

# Create our phase-out schedule from 2024 (100%) to 2040 (0%)
phase_out_years = range(2024, 2041)
phase_out_factors = np.linspace(1.0, 0.0, len(phase_out_years))

# Create a new DataFrame with phase-out
phased_allocation = industrial_data.copy()

# Apply the phase-out factors
for i, year in enumerate(phase_out_years):
    if year in phased_allocation.index:
        phased_allocation.loc[year, 'baseline_allocation'] = industrial_data.loc[year, 'baseline_allocation'] * phase_out_factors[i]

# Set any years after 2040 to zero
for year in phased_allocation.index:
    if year > 2040:
        phased_allocation.loc[year, 'baseline_allocation'] = 0.0

# Apply the industrial allocation phase-out to ALL sensitivity levels
for i in range(len(NZU.scenarios)):
    NZU.set_series('baseline_allocation', phased_allocation['baseline_allocation'], 'industrial', scenario_index=i)

# Update the industrial allocation series for ALL sensitivity levels
for i in range(len(NZU.scenarios)):
    NZU.set_series('baseline_allocation', phased_allocation['baseline_allocation'], 'industrial', scenario_index=i)
    print(f"Applied industrial allocation phase-out to scenario {i} ({NZU.scenarios[i]})")


[DEBUG] DataHandler: Loading industrial allocation data for config 'central'
[DEBUG] DataHandler: Filtering industrial allocation data for config 'central'

[DEBUG] DataHandler: Loading industrial allocation data for config 'central'
[DEBUG] DataHandler: Filtering industrial allocation data for config 'central'
Updated industrial.baseline_allocation for scenario '95% Lower'

[DEBUG] DataHandler: Loading industrial allocation data for config 'central'
[DEBUG] DataHandler: Filtering industrial allocation data for config 'central'
Updated industrial.baseline_allocation for scenario '1 s.e lower'

[DEBUG] DataHandler: Loading industrial allocation data for config 'central'
[DEBUG] DataHandler: Filtering industrial allocation data for config 'central'
Updated industrial.baseline_allocation for scenario 'central'

[DEBUG] DataHandler: Loading industrial allocation data for config 'central'
[DEBUG] DataHandler: Filtering industrial allocation data for config 'central'
Updated industrial.base

In [6]:
# Print the modified industrial allocation to verify
print("\nModified industrial allocation with phase-out by 2040:")
print(phased_allocation.loc[[2024, 2030, 2035, 2040, 2045]])


Modified industrial allocation with phase-out by 2040:
      baseline_allocation  activity_adjustment
year                                          
2024          6058.110998                  1.0
2030          2928.459897                  1.0
2035          1276.728037                  1.0
2040             0.000000                  1.0
2045             0.000000                  1.0


## 4. Run the Model with Uncertainty Analysis

Now let's run the model with our configured range scenario.

In [7]:
# Run the model
results = NZU.run()


[DEBUG] Starting range scenario configuration
[DEBUG] Storing existing component configurations
[DEBUG] Stored config for scenario 0: {'forestry': 'low', 'auctions': 'central', 'industrial_allocation': 'central', 'emissions': 'central', 'stockpile': 'central', 'demand_sensitivity': '95% Lower', 'demand_model_number': 2}
[DEBUG] Stored config for scenario 1: {'forestry': 'low', 'auctions': 'central', 'industrial_allocation': 'central', 'emissions': 'central', 'stockpile': 'central', 'demand_sensitivity': '1 s.e lower', 'demand_model_number': 2}
[DEBUG] Stored config for scenario 2: {'forestry': 'low', 'auctions': 'central', 'industrial_allocation': 'central', 'emissions': 'central', 'stockpile': 'central', 'demand_sensitivity': 'central', 'demand_model_number': 2}
[DEBUG] Stored config for scenario 3: {'forestry': 'low', 'auctions': 'central', 'industrial_allocation': 'central', 'emissions': 'central', 'stockpile': 'central', 'demand_sensitivity': '1 s.e upper', 'demand_model_number': 

## 5. Generate Range Charts

One of the key advantages of range runs is the ability to generate charts that show uncertainty bands around central estimates. Let's explore this capability.

In [8]:
# Verify that both forestry and industrial allocation configurations were applied correctly
print("\n===== VERIFICATION OF MODEL CONFIGURATIONS =====")

# 1. Verify forestry configuration by checking forestry supply for 2030
print("\nForestry Supply Values for 2030 (should reflect 'low' config for all sensitivity levels):")
forestry_supply = NZU.supply.xs('forestry', level='variable', axis=1)
if 2050 in forestry_supply.index:
    for scenario in NZU.scenarios:
        print(f"{scenario}: {forestry_supply.loc[2050, scenario]:.2f} kt CO₂-e")
else:
    print("Year 2030 not found in forestry supply data")

# 2. Verify industrial allocation phase-out around 2040
print("\nIndustrial Allocation Values near phase-out (2038-2042):")
industrial_allocation = NZU.supply.xs('industrial', level='variable', axis=1)
for year in range(2038, 2043):
    if year in industrial_allocation.index:
        print(f"\nYear {year}:")
        for scenario in NZU.scenarios:
            print(f"{scenario}: {industrial_allocation.loc[year, scenario]:.2f} kt CO₂-e")
    else:
        print(f"Year {year} not found in industrial allocation data")


===== VERIFICATION OF MODEL CONFIGURATIONS =====

Forestry Supply Values for 2030 (should reflect 'low' config for all sensitivity levels):
95% Lower: 14489.65 kt CO₂-e
1 s.e lower: 14489.65 kt CO₂-e
central: 14489.65 kt CO₂-e
1 s.e upper: 14489.65 kt CO₂-e
95% Upper: 14489.65 kt CO₂-e

Industrial Allocation Values near phase-out (2038-2042):

Year 2038:
95% Lower: 465.69 kt CO₂-e
1 s.e lower: 465.69 kt CO₂-e
central: 465.69 kt CO₂-e
1 s.e upper: 465.69 kt CO₂-e
95% Upper: 465.69 kt CO₂-e

Year 2039:
95% Lower: 225.35 kt CO₂-e
1 s.e lower: 225.35 kt CO₂-e
central: 225.35 kt CO₂-e
1 s.e upper: 225.35 kt CO₂-e
95% Upper: 225.35 kt CO₂-e

Year 2040:
95% Lower: 0.00 kt CO₂-e
1 s.e lower: 0.00 kt CO₂-e
central: 0.00 kt CO₂-e
1 s.e upper: 0.00 kt CO₂-e
95% Upper: 0.00 kt CO₂-e

Year 2041:
95% Lower: 0.00 kt CO₂-e
1 s.e lower: 0.00 kt CO₂-e
central: 0.00 kt CO₂-e
1 s.e upper: 0.00 kt CO₂-e
95% Upper: 0.00 kt CO₂-e

Year 2042:
95% Lower: 0.00 kt CO₂-e
1 s.e lower: 0.00 kt CO₂-e
central: 0.00 

In [9]:
# Initialise chart generator
chart_gen = ChartGenerator(NZU)

# Generate carbon price chart with uncertainty bands
print("Generating carbon price chart with uncertainty bands...")
price_chart = chart_gen.carbon_price_chart()
price_chart.update_layout(title="Carbon Price Projection with Uncertainty Bands")
price_chart.show()
price_chart.write_image(str(output_dir / "range_price_chart.png"))

# Generate emissions pathway chart with uncertainty bands
print("\nGenerating emissions pathway chart with uncertainty bands...")
emissions_chart = chart_gen.emissions_pathway_chart()
emissions_chart.update_layout(title="Emissions Pathway with Uncertainty Bands")
emissions_chart.show()
emissions_chart.write_image(str(output_dir / "range_emissions_chart.png"))

# Generate stockpile balance chart with uncertainty bands
print("\nGenerating stockpile balance chart with uncertainty bands...")
stockpile_chart = chart_gen.stockpile_balance_chart()
stockpile_chart.update_layout(title="Stockpile Balance with Uncertainty Bands")
stockpile_chart.show()
stockpile_chart.write_image(str(output_dir / "range_stockpile_chart.png"))

Generating carbon price chart with uncertainty bands...



Generating emissions pathway chart with uncertainty bands...



Generating stockpile balance chart with uncertainty bands...


## 6. Create Interactive HTML Dashboard

NZUpy provides functionality to create interactive HTML dashboards with multiple charts. Let's create one for our range run.

In [10]:
# Create a collection of charts for our dashboard
chart_collection = {
    "Carbon Price Projection": price_chart,
    "Emissions Pathway": emissions_chart,
    "Stockpile Balance": stockpile_chart
}

# Create an interactive HTML dashboard
dashboard_html = create_chart_page(
    charts=chart_collection,
    title="NZUpy Range Run Analysis",
    subtitle="Uncertainty Analysis with Industrial Phase-out and Low Forestry",
    output_dir=output_dir,
    filename="range_run_dashboard.html"
)

print(f"\nInteractive dashboard saved to: {output_dir / 'range_run_dashboard.html'}")
print("Open this file in a web browser to explore the interactive charts")

Chart page saved to: c:\Users\nzkri\Python projects\NZUpy\examples\outputs\03_range_runs\range_run_dashboard.html

Interactive dashboard saved to: c:\Users\nzkri\Python projects\NZUpy\examples\outputs\03_range_runs\range_run_dashboard.html
Open this file in a web browser to explore the interactive charts


## 7. Extract and Analyse Uncertainty Ranges

Let's extract the data and analyse the uncertainty ranges for some key metrics.

In [11]:
# Extract carbon price data for all sensitivity levels
price_data = NZU.prices.xs('carbon_price', level='variable', axis=1)

# Focus on a selection of years for each demand sensitivity scenario
forecast_years = [2030, 2035, 2040, 2045, 2050]

print("\nUncertainty ranges for carbon prices ($/tCO2e):")
print(f"{'Year':<10} {'Central':<10} {'±1 Std Err':<15} {'95% Range':<20}")
print("-" * 55)

for year in forecast_years:
    if year in price_data.index:
        central = price_data['central'][year]
        lower_std = price_data['1 s.e lower'][year]
        upper_std = price_data['1 s.e upper'][year]
        lower_95 = price_data['95% Lower'][year]
        upper_95 = price_data['95% Upper'][year]
        
        std_range = f"[${upper_std:.1f} - ${lower_std:.1f}]"
        conf_range = f"[${upper_95:.1f} - ${lower_95:.1f}]"
        
        print(f"{year:<10} ${central:<10.1f} {std_range:<15} {conf_range:<20}")

# Now do the same for emissions
emissions_data = NZU.demand.xs('emissions', level='variable', axis=1)

print("\nUncertainty ranges for emissions (kt CO2e):")
print(f"{'Year':<10} {'Central':<10} {'±1 Std Err':<15} {'95% Range':<20}")
print("-" * 55)

for year in forecast_years:
    if year in emissions_data.index:
        central = emissions_data['central'][year]
        lower_std = emissions_data['1 s.e lower'][year]
        upper_std = emissions_data['1 s.e upper'][year]
        lower_95 = emissions_data['95% Lower'][year]
        upper_95 = emissions_data['95% Upper'][year]
        
        std_range = f"[{upper_std:,.0f} - {lower_std:,.0f}]"
        conf_range = f"[{upper_95:,.0f} - {lower_95:,.0f}]"
        
        print(f"{year:<10} {central:<10,.0f} {std_range:<15} {conf_range:<20}")



Uncertainty ranges for carbon prices ($/tCO2e):
Year       Central    ±1 Std Err      95% Range           
-------------------------------------------------------
2030       $82.6       [$82.6 - $82.6] [$75.3 - $82.6]     
2035       $104.9      [$104.9 - $104.9] [$88.6 - $104.9]    
2040       $133.3      [$133.3 - $133.3] [$104.2 - $133.3]   
2045       $169.3      [$169.3 - $169.3] [$122.6 - $169.3]   
2050       $215.1      [$215.1 - $215.1] [$144.2 - $215.1]   

Uncertainty ranges for emissions (kt CO2e):
Year       Central    ±1 Std Err      95% Range           
-------------------------------------------------------
2030       30,160     [28,619 - 30,990] [26,496 - 31,441]   
2035       25,176     [22,564 - 26,488] [19,168 - 27,157]   
2040       21,094     [17,790 - 22,714] [14,033 - 23,525]   
2045       17,876     [13,816 - 19,834] [9,869 - 20,799]    
2050       14,965     [9,993 - 17,326] [5,926 - 18,472]    


As we can see with above, varying demand response elasticity assumptions can affect price and gross emissions outcomes, with a more elastic demand response seeing lower emissions and a lower carbon price.

## 8. Exporting results data for later use

NZUpy also provides helper functions to explore available variables and export results for further analysis. 

First, let's see what variables are available in our results, using a helper `list_variables` function:

In [12]:
# Create an output formatter instance
from model.utils.output_format import OutputFormat
output_formatter = OutputFormat(NZU)

# List all available variables by category
output_formatter.list_variables()

Available variables by category:

Prices:
  self.prices - Carbon prices by scenario
  Access real prices: model.prices.xs('carbon_price', level='variable', axis=1)
  Access nominal prices: model.prices.xs('carbon_price_nominal', level='variable', axis=1)
  Example for specific scenario: model.prices[('central', 'carbon_price')]
  Example for nominal prices: model.prices[('central', 'carbon_price_nominal')]

Core model outputs:
  self.core.xs('avg_price_change_rate', level='variable', axis=1) - Average annual price growth (%)
  self.core.xs('price_change_rate', level='variable', axis=1) - Annual price growth rate (%)
  self.core.xs('supply_demand_balance', level='variable', axis=1) - Supply minus demand by year (kt CO₂-e)

Supply components:
  self.supply.xs('auction', level='variable', axis=1) - Units supplied through auction (kt CO₂-e)
  self.supply.xs('forestry', level='variable', axis=1) - Units from forestry removals (kt CO₂-e)
  self.supply.xs('industrial', level='variable', axis=

We can check and export a single variable if we wsh, lets check the head of our data for the ratio of stockpile size to annual demand.

In [13]:
# Get the stockpile ratio to demand for all scenarios
stockpile_ratio = NZU.stockpile.xs('ratio_to_demand', level='variable', axis=1)

# Display the first few rows to check the data
print("\nStockpile ratio to demand (first & last 5 years):")
print(stockpile_ratio.head(10).round(2))


Stockpile ratio to demand (first & last 5 years):
scenario  95% Lower  1 s.e lower  central  1 s.e upper  95% Upper
year                                                             
2024           3.76         3.76     3.76         3.77       3.77
2025           3.22         3.22     3.23         3.23       3.24
2026           2.76         2.77     2.78         2.81       2.83
2027           2.33         2.35     2.38         2.43       2.50
2028           2.02         2.05     2.09         2.16       2.27
2029           1.72         1.75     1.80         1.88       2.03
2030           1.57         1.59     1.65         1.75       1.94
2031           1.45         1.48     1.57         1.76       2.10
2032           1.40         1.43     1.52         1.77       2.28
2033           1.41         1.45     1.54         1.81       2.52


Interesting, we see a decreasing stockpile size relative to annual demand at most demand sensitivity levels over this time period, though this trend starts to reverse from the early 2030s in our higher elasticity scenarios (1 s.e upper & 95% upper). We can export this to a CSV for later use too.

In [14]:
# Create a DataFrame with first 10 years as index and scenarios as columns
export_df = pd.DataFrame(index=stockpile_ratio.index[:10])
export_df.index.name = 'year'  # Name the index column

# Add the stockpile ratio data for each scenario
for scenario in NZU.scenarios:
    export_df[f'stockpile_ratio_{scenario}'] = stockpile_ratio[scenario][:10].round(2)

#Export to CSV
csv_path = output_dir / 'stockpile_ratio_to_annual_demand.csv'
export_df.to_csv(csv_path)

Or we can batch save our results data to CSV files. Lets place these in a subfolder for safekeeping. 

In [15]:
# Export all model data to separate CSV files
print("\nExporting all model data to separate CSV files...")
chart_gen.export_csv_data(output_dir=output_dir / 'csv_data')


Exporting all model data to separate CSV files...
Exported prices data to c:\Users\nzkri\Python projects\NZUpy\examples\outputs\03_range_runs\csv_data\prices.csv
Exported core data to c:\Users\nzkri\Python projects\NZUpy\examples\outputs\03_range_runs\csv_data\core.csv
Error exporting supply data: [Errno 13] Permission denied: 'c:\\Users\\nzkri\\Python projects\\NZUpy\\examples\\outputs\\03_range_runs\\csv_data\\supply.csv'
Exported demand data to c:\Users\nzkri\Python projects\NZUpy\examples\outputs\03_range_runs\csv_data\demand.csv
Exported stockpile data to c:\Users\nzkri\Python projects\NZUpy\examples\outputs\03_range_runs\csv_data\stockpile.csv
Exported auctions data to c:\Users\nzkri\Python projects\NZUpy\examples\outputs\03_range_runs\csv_data\auctions.csv
Exported industrial data to c:\Users\nzkri\Python projects\NZUpy\examples\outputs\03_range_runs\csv_data\industrial.csv
Exported forestry data to c:\Users\nzkri\Python projects\NZUpy\examples\outputs\03_range_runs\csv_data\fo

## 9. Conclusion

This notebook has demonstrated NZUpy's capability to perform uncertainty analysis through range runs. We've seen how to:

1. **Set up a range run**
2. **Configure custom policy settings** with industrial allocation phased-out more quickly and low forestry
3. **Generate and interpret range charts** with uncertainty bands
4. **Create interactive dashboards** for exploring results
5. **Extract and analyse uncertainty ranges** for key metrics
6. **Build custom visualisations** to highlight specific aspects of the results

Range runs are a powerful tool for robust policy analysis, allowing stakeholders to understand not just the expected outcomes of different settings but also the uncertainty around those projections. This approach provides a more complete picture for decision-making in the complex and evolving landscape of emissions trading.

The interactive nature of NZUpy's visualisations enhances the ability to explore and understand these complex results, making it easier to communicate findings to stakeholders.